# Exploring two dimer toy model with (symmetric) RBM

In [112]:
import netket as nk
import numpy as np
import time
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import jax
import flax
import pprint
print("NetKet version: {}".format(nk.__version__))
print("NumPy version: {}".format(np.__version__))

NetKet version: 3.1.2
NumPy version: 1.20.3


Setup relevant parameters

In [113]:
"""lattice"""
SITES    = 4            # 4, 8, 16, 20 ... number of vertices in a tile determines the tile shape 
JEXCH1   = 1            # nn interaction
JEXCH2   = 1            # nnn interaction
#USE_MSR  = True        # should we use a Marshall sign rule?
"""machine learning"""
TOTAL_SZ = 0         # 0, None ... restriction of Hilbert space
DTYPE = np.complex128   # type of weights in neural network
SAMPLER = 'exact'       # 'local' = MetropolisLocal, 'exchange' = MetropolisExchange
ALPHA = 16              # N_hidden / N_visible
ETA   = .01             # learning rate (0.01 usually works)
SAMPLES = 2000
NUM_ITER = 500

OUT_NAME = "SS-RBM_ops"+str(SITES)+"j1="+str(JEXCH1) # output file name

Lattice and hamiltonian definition: &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $ H = J_{1} \sum\limits_{\langle i,j \rangle}^{L} \vec{\sigma}_{i} \cdot \vec{\sigma}_{j} + J_{2} \sum\limits_{\langle\langle i,j \rangle\rangle_{SS}}^{L}  \vec{\sigma}_{i} \cdot \vec{\sigma}_{j}\,, $

In [114]:
# Define custom graph
edge_colors = []

edge_colors.append([0,1,1])
edge_colors.append([2,3,1])
# SS 2x2 lattice:
# edge_colors.append([1,2,1])
# edge_colors.append([0,3,1])
# edge_colors.append([0,2,2])
# edge_colors.append([1,3,2])

# Define the netket graph object
g = nk.graph.Graph(edges=edge_colors) #,n_nodes=3)
N = g.n_nodes

hilbert = nk.hilbert.Spin(s=.5, N=g.n_nodes, total_sz=TOTAL_SZ)


#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz)) #=sz*sz
#Exchange interactions
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]]) #=sx*sx+sy*sy = 1/2*(sp*sm+sm*sp)
full_spin = mszsz+exchange # = S*S = sx*sx + sy*sy + sz*sz
bond_color = [1, 2, 1, 2]

In [115]:
translations = []
for perm in g.automorphisms():
    aperm = np.asarray(perm)
    if (aperm[0],aperm[1],aperm[3]) in ((0,1,3),(2,3,1)): # N = 16, two dimmers with just translation
    # if (aperm[0],aperm[1],aperm[3]) in ((0,1,3),(3,2,0),(2,3,1),(1,0,2)): # N = 16, two dimmers plus second translation option
        translations.append(nk.utils.group._permutation_group.Permutation(aperm))
# print(translations)
translation_group = nk.utils.group._permutation_group.PermutationGroup(translations,degree=SITES)
print("Full symmetries:")
print(len(g.automorphisms()))
print(g.automorphisms())
print("Restricted symmetries:")
print(len(translation_group))
print(translation_group)

Full symmetries:
8
PermutationGroup(elems=[Permutation([0, 1, 2, 3]), Permutation([0, 1, 3, 2]), Permutation([1, 0, 2, 3]), Permutation([1, 0, 3, 2]), Permutation([2, 3, 0, 1]), Permutation([2, 3, 1, 0]), Permutation([3, 2, 0, 1]), Permutation([3, 2, 1, 0])], degree=4)
Restricted symmetries:
2
PermutationGroup(elems=[Permutation([0, 1, 2, 3]), Permutation([2, 3, 0, 1])], degree=4)


## Hamoltonian

In [116]:
bond_operator = [
    (JEXCH1 * mszsz).tolist(),
    (JEXCH2 * mszsz).tolist(),
    (JEXCH1 * exchange).tolist(), # minus in case of MSR
    (JEXCH2 * exchange).tolist(),
]
bond_operatorMSR = [
    (JEXCH1 * mszsz).tolist(),
    (JEXCH2 * mszsz).tolist(),
    (-JEXCH1 * exchange).tolist(), # minus in case of MSR
    (JEXCH2 * exchange).tolist(),
]
ha = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)
ha_MSR = nk.operator.GraphOperator(hilbert, graph=g, bond_ops=bond_operatorMSR, bond_ops_colors=bond_color)


## Exact diagonalization

In [117]:
if g.n_nodes < 21:
    start = time.time()
    evals, eigvects = nk.exact.lanczos_ed(ha, k=3, compute_eigenvectors=True)
    #evals = nk.exact.lanczos_ed(ha, compute_eigenvectors=False) #.lanczos_ed
    end = time.time()
    diag_time = end - start
    exact_ground_energy = evals[0]
    print("Ground state energy:",exact_ground_energy, "\nIt took ", round(diag_time,2), "s =", round((diag_time)/60,2),"min")
else:
    print("System is too large for exact diagonalization. Setting exact_ground_energy = 0 (which is wrong)")
    exact_ground_energy = [0,0,0]
    eigvects = None 
# -36.2460684609957 

Ground state energy: -6.0000000000000036 
It took  0.0 s = 0.0 min


## RBM

!!POZOZ je použito `g.automorphisms()` namísto `g.translations()` - obsahuje to tedy symetrii zrcadlení řetízku navíc, a proto na konci dostaneme jen dvě sady filtrů namísto čtyřech.

Vskutku, metoga g.automorphisms() najde veškeré symetrie. V tomto případě jsou všechny složeny z nezávislých symetrií:
<!-- ### Heisenberg
 * zrcadlení okolo $x$, 
 * zdcadlení okolo $y$, 
 * jedna rotace o $90$ stupňů (ostatní se dají složit se zrcadleními),
 * ``L_NUM*L_NUM`` translací,

tedy celkem $8L_{NUM}^2$ (pro $L_{NUM} > 2$, protože pro $L_{NUM}=2$ je zrcadlení a translace ekvivalentní) -->

### Shastry-Sutherland 
 * ``L_NUM*L_NUM`` translací,
 * rotace o $90$ stupňů,

tedy celkem $4L_{NUM}^2$ (stejně je vždy $L_{NUM} \geq 4$)

```
#comment from github: Have you tried using g.translation_group() (or g.translations() is some of the previous beta versions)
 instead of .automorphisms() to check whether that makes a difference? (Using the larger group of all 6x6 space group symmetries 
 is probably also the reason why a large alpha is required in your version.) While symmetries should make a state easier to learn,
 I do recall that using the full space or automorphism group (instead of the usually easier-to-handle translation group) can 
 sometimes introduce convergence issues (which at least need some more careful tuning of optimization settings and probably 
 initial params to overcome).
#https://github.com/netket/netket/discussions/838
```

In [118]:
#definice modelu, sampleru atd.

#Feature dimensions of hidden layers, from first to last
feature_dims = (8,8,8,8)

#Number of layers
num_layers = 4

#Define the GCNN
# machine = nk.models.GCNN(symmetries = g.automorphisms(), layers = num_layers, features = feature_dims)
# machine_MSR = nk.models.GCNN(symmetries = g.automorphisms(), layers = num_layers, features = feature_dims)

# Symmetric RBM Spin Machine
# machine = nk.models.RBM(dtype=DTYPE, alpha=ALPHA) 
machine = nk.models.RBMSymm(translation_group, dtype=DTYPE, alpha=ALPHA) 
# machine = nk.models.RBMSymm(g.automorphisms(), dtype=DTYPE, alpha=ALPHA) 

print("Number of automorphisms:", len(g.automorphisms()))
# Symmetric RBM Spin Machine with MSR
# machine_MSR = nk.models.RBM(dtype=DTYPE, alpha=ALPHA)
machine_MSR = nk.models.RBMSymm(translation_group, dtype=DTYPE, alpha=ALPHA)
# machine_MSR = nk.models.RBMSymm(g.automorphisms(), dtype=DTYPE, alpha=ALPHA)


# Meropolis Exchange Sampling
if SAMPLER == 'local':
    sampler = nk.sampler.MetropolisLocal(hilbert=hilbert)
    sampler_MSR = nk.sampler.MetropolisLocal(hilbert=hilbert)
else:
    sampler = nk.sampler.MetropolisExchange(hilbert=hilbert, graph=g)
    sampler_MSR = nk.sampler.MetropolisExchange(hilbert=hilbert, graph=g)
    if SAMPLER != 'exchange':
        print("Warning! Undefined sampler:", SAMPLER, ", dafaulting to MetropolisExchange sampler")


# Optimzer
optimizer = nk.optimizer.Sgd(learning_rate=ETA)
optimizer_MSR = nk.optimizer.Sgd(learning_rate=ETA)

# Stochastic Reconfiguration
sr  = nk.optimizer.SR(diag_shift=0.1)
sr_MSR  = nk.optimizer.SR(diag_shift=0.1)

# The variational state (drive to byla nk.variational.MCState)
vss = nk.vqs.MCState(sampler, machine, n_samples=SAMPLES)
vs_MSR  = nk.vqs.MCState(sampler_MSR, machine_MSR, n_samples=SAMPLES)
vss.init_parameters(jax.nn.initializers.normal(stddev=0.001))
vs_MSR.init_parameters(jax.nn.initializers.normal(stddev=0.001))


gs_normal = nk.VMC(hamiltonian=ha ,optimizer=optimizer,preconditioner=sr,variational_state=vss)               # 0 ... symmetric
gs_MSR = nk.VMC(hamiltonian=ha_MSR ,optimizer=optimizer_MSR,preconditioner=sr_MSR,variational_state=vs_MSR)   # 1 ... symmetric+MSR

Number of automorphisms: 8
Warning! Undefined sampler: exact , dafaulting to MetropolisExchange sampler


# Operators

In [119]:
def SS(i,j): #different method of definition
    if i==j:
        return nk.operator.LocalOperator(hilbert,operators=[[3,0],[0,3]],acting_on=[i])
    else:
        return nk.operator.LocalOperator(hilbert,operators=(mszsz+exchange),acting_on=[i,j])

def P(i,j,msr=False): # two particle permutation operator
    if msr == False:
        return .5*(SS(i,j)+nk.operator.LocalOperator(hilbert,operators=[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]],acting_on=[i,j]))
    else:
        raise NotImplementedError()


# Calculation

In [130]:
no_of_runs = 1 #2 ... bude se pocitat i druhý způsob (za použití MSR)
use_MSR = 0 # in case of one run
NUM_ITER = 1000
print("J_1 =", JEXCH1)
if exact_ground_energy != 0:
    print("Expected exact energy:", exact_ground_energy)
for i,gs in enumerate([gs_normal,gs_MSR][use_MSR:use_MSR+no_of_runs]):
    start = time.time()
    #gs.info()
    gs.run(out=OUT_NAME+str(i), n_iter=int(NUM_ITER), obs={'symmetry':P(0,1)})#,obs={'DS_factor': m_dimer_op})#,'PS_factor':m_plaquette_op,'AF_factor':m_s2_op})
    end = time.time()
    print("The type {} of RBM calculation took {} min".format(i, (end-start)/60))


J_1 = 1
Expected exact energy: -6.0000000000000036


100%|██████████| 1000/1000 [01:48<00:00,  9.20it/s, Energy=nan+nanj ± nan [σ²=nan]]

The type 0 of RBM calculation took 1.8135178367296854 min


In [131]:
print(gs_normal.estimate(P(2,3)@P(0,1)))

nan+nanj ± nan [σ²=nan]


## Energy

In [122]:
# exact energy line
figure = go.Figure(
    data=[go.Scatter(x=(0,NUM_ITER),y=(exact_ground_energy,exact_ground_energy),mode="lines",line=go.scatter.Line(color="#000000",width=1), name="exact energy")], 
    layout=go.Layout(template="simple_white",
        xaxis=dict(title="Iteration", mirror=True, showline=True),
        yaxis=dict(title="Energy", mirror=True, showline=True),
        title=("<b>"+"S-S"+" model </b>, L="+str(SITES)+", J2 ="+str(JEXCH2)+ ", J1 ="+str(JEXCH1)+" , η="+str(ETA)+", α="+str(ALPHA)+", samples="+str(SAMPLES))))

In [132]:
# import the data from log file
OUT_NAME_suffixless=OUT_NAME
data = []
for i in range(no_of_runs):
    data.append(json.load(open(OUT_NAME_suffixless+str(i)+".log")))
names = ["normal basis","MSR basis"]
if DTYPE==np.complex128:# and False:
    energy_convergence = [data[i]["Energy"]["Mean"]["real"] for i in range(no_of_runs)]
    symmetry = [data[i]["symmetry"]["Mean"]["real"] for i in range(no_of_runs)]
else:
    energy_convergence = [data[i]["Energy"]["Mean"] for i in range(no_of_runs)]
    symmetry = [data[i]["symmetry"]["Mean"] for i in range(no_of_runs)]
for i in range(no_of_runs):
    figure.add_trace(go.Scatter(
        x=data[i]["Energy"]["iters"], y=energy_convergence[i],
        name=names[i]
    ))
    figure.add_trace(go.Scatter(
        x=data[i]["Energy"]["iters"], y=symmetry[i],
        name=names[i]+"_swap"
    ))

#figure.add_hline(y=exact_gs_energy)
figure.update_layout(xaxis_title="Iteration",yaxis_title="Energy")
figure.show()

# Calculating symmetrizations

In [124]:
alpha = 2
perms = np.array([[0,1,2],[1,0,2]])#[[0,1,2,3],[0,1,3,2],[1,0,2,3],[1,0,3,2]])
n_symm, n_sites = perms.shape
features = int(alpha*n_sites/n_symm)
n_hidden = features * n_symm
ij = np.arange(n_sites * n_hidden)
i, j = np.unravel_index(ij, (n_sites, n_hidden))
k = perms[j%n_symm,i]
l = np.floor_divide(j, n_symm)
kl = np.ravel_multi_index((k,l),(n_sites,features))
print(kl, features)

[0 3 1 4 2 5 3 0 4 1 5 2 6 6 7 7 8 8] 3


In [125]:
for i in range(9):
    print(np.where(kl==i))

(array([0, 7]),)
(array([2, 9]),)
(array([ 4, 11]),)
(array([1, 6]),)
(array([3, 8]),)
(array([ 5, 10]),)
(array([12, 13]),)
(array([14, 15]),)
(array([16, 17]),)
